# ECC-Based Hybrid Encryption / Decryption

# 1. ECC + AES-GCM hybrid encryption/decryption

In [1]:

from tinyec import registry
from Cryptodome.Cipher import AES
import hashlib, secrets, binascii

def encrypt_AES_GCM(msg, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM)
    ciphertext, authTag = aesCipher.encrypt_and_digest(msg)
    return (ciphertext, aesCipher.nonce, authTag)
#The ciphertext is obtained by the symmetric AES-GCM encryption
#nonce (random AES initialization vector) and authTag 
#(the MAC code of the encrypted text, obtained by the GCM block mode)


def decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce)
    plaintext = aesCipher.decrypt_and_verify(ciphertext, authTag)
    return plaintext

def ecc_point_to_256_bit_key(point):
    sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big'))
    sha.update(int.to_bytes(point.y, 32, 'big'))
    return sha.digest()

curve = registry.get_curve('brainpoolP256r1')


def encrypt_ECC(msg, pubKey):
    AlicePrivKey = secrets.randbelow(curve.field.n)
# sharedECCKey is a point
    sharedECCKey = AlicePrivKey * pubKey
# convert 256 bits secretKey for AES scheme
    secretKey = ecc_point_to_256_bit_key(sharedECCKey)
    
    ciphertext, nonce, authTag = encrypt_AES_GCM(msg, secretKey)
    
# a randomly generated ephemeral public key ciphertextPubKey, 
#which will be encapsulated in the encrypted message and will be
#used to recover the AES symmetric key during the decryption 
    AlicePubKey = AlicePrivKey * curve.g
    return (ciphertext, nonce, authTag, AlicePubKey)


def decrypt_ECC(encryptedMsg, privKey):
    (ciphertext, nonce, authTag, AlicePubKey) = encryptedMsg
    sharedECCKey = privKey * AlicePubKey
    secretKey = ecc_point_to_256_bit_key(sharedECCKey)
    print(len(secretKey))
    plaintext = decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey)
    return plaintext


msg = b'I really want a border collie'
print("original msg:", msg)
print("\n")
BobPrivKey = secrets.randbelow(curve.field.n)
BobPubKey = BobPrivKey * curve.g

encryptedMsg = encrypt_ECC(msg, BobPubKey)
encryptedMsgObj = {
    'ciphertext': binascii.hexlify(encryptedMsg[0]),
    'nonce': binascii.hexlify(encryptedMsg[1]),
    'authTag': binascii.hexlify(encryptedMsg[2]),
    'AlicePubKey': hex(encryptedMsg[3].x) + hex(encryptedMsg[3].y % 2)[2:]
}
print("encrypted msg:", encryptedMsgObj)

print("\n")
decryptedMsg = decrypt_ECC(encryptedMsg, BobPrivKey)
print("decrypted msg:", decryptedMsg)

original msg: b'I really want a border collie'


encrypted msg: {'ciphertext': b'567681761cbe48f510d267cfe06f19394ccc53879ccccf62ef3f99bba5', 'nonce': b'b517f63b2f23c7cad6528ba5a9d407a8', 'authTag': b'e5a0876400467f5c3c641d2f95ff36a6', 'AlicePubKey': '0x83588ae382706d04066bbb644868cb451ef1f9c7592c3935695f9b5313f6a1de0'}


32
decrypted msg: b'I really want a border collie'


# 2. ECC + AES-CTR hybrid encryption/decryption

In [2]:
import pyaes, binascii, os, secrets

def encrypt_AES_CTR(msg,key,iv):
# Encrypt the plaintext with the given key:
#   ciphertext = AES-256-CTR-Encrypt(plaintext, key, iv)
    aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
    ciphertext = aes.encrypt(msg)
    return ciphertext

def decrypt_AES_CTR(ciphertext,key,iv):
# Decrypt the ciphertext with the given key:
#   plaintext = AES-256-CTR-Decrypt(ciphertext, key, iv)
    aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
    decrypted = aes.decrypt(ciphertext)
    return  decrypted

def ecc_point_to_256_bit_key(point):
    sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big'))
    sha.update(int.to_bytes(point.y, 32, 'big'))
    return sha.digest()

curve = registry.get_curve('brainpoolP256r1')

def encrypt_ECC(msg, pubKey):
    AlicePrivKey = secrets.randbelow(curve.field.n)
# sharedECCKey is a point
    sharedECCKey = AlicePrivKey * pubKey
# convert 256 bits secretKey for AES scheme
    secretKey = ecc_point_to_256_bit_key(sharedECCKey)
    print('AES encryption key:', binascii.hexlify(secretKey))
    iv = secrets.randbits(256)
    ciphertext = encrypt_AES_CTR(msg,secretKey,iv)
# a randomly generated ephemeral public key ciphertextPubKey, 
#which will be encapsulated in the encrypted message and will be
#used to recover the AES symmetric key during the decryption 
    AlicePubKey = AlicePrivKey * curve.g
    return (ciphertext, AlicePubKey,iv)


def decrypt_ECC(encryptedMsg, privKey):
    (ciphertext,AlicePubKey,iv) = encryptedMsg
    sharedECCKey = privKey * AlicePubKey
    secretKey = ecc_point_to_256_bit_key(sharedECCKey)
    decrypted = decrypt_AES_CTR(ciphertext,secretKey,iv)
    return decrypted


msg = b'I really want a border collie'
print("original msg:", msg)
print("\n")
BobPrivKey = secrets.randbelow(curve.field.n)
BobPubKey = BobPrivKey * curve.g

encryptedMsg = encrypt_ECC(msg, BobPubKey)
encryptedMsgObj = {
    'ciphertext': binascii.hexlify(encryptedMsg[0]),
    'AlicePubKey': hex(encryptedMsg[1].x) + hex(encryptedMsg[1].y % 2)[2:],
    'iv': encryptedMsg[2]
}

print("encrypted msg:", encryptedMsgObj)

print("\n")
decryptedMsg = decrypt_ECC(encryptedMsg, BobPrivKey)
print("decrypted msg:", decryptedMsg)  

original msg: b'I really want a border collie'


AES encryption key: b'84c3edd67c136f1f251031b790c51e9ae7f253c22afb8e30a7bcd8b578b7a872'
encrypted msg: {'ciphertext': b'f78426fbe913acbbfc5d256b4f4093f001779577158363d00b5768bea2', 'AlicePubKey': '0x3589147a058270f1ca650cab49bd5a12e675d12bbdfa205c5a631821e4bb3b440', 'iv': 100132564261943486725353642781655798137386724996805482301239804156948240295541}


decrypted msg: b'I really want a border collie'


# 3. ECC + Chacha20 hybrid encryption/decryption

In [3]:
import pyaes, binascii, os, secrets
from chacha20poly1305 import ChaCha20Poly1305

def encrypt_ChaCha20(msg,key,nonce):
    cip = ChaCha20Poly1305(key)
    ciphertext = cip.encrypt(nonce, msg)
    return ciphertext

def decrypt_ChaCha20(key,ciphertext,nonce):
    cip = ChaCha20Poly1305(key)
    decrypted = cip.decrypt(nonce, ciphertext)
    return  decrypted

def ecc_point_to_256_bit_key(point):
    sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big'))
    sha.update(int.to_bytes(point.y, 32, 'big'))
    return sha.digest()

curve = registry.get_curve('brainpoolP256r1')

def encrypt_ECC(msg, pubKey):
    AlicePrivKey = secrets.randbelow(curve.field.n)
# sharedECCKey is a point
    sharedECCKey = AlicePrivKey * pubKey
# convert 256 bits secretKey
    secretKey = ecc_point_to_256_bit_key(sharedECCKey)
    print('secret key:', binascii.hexlify(secretKey))
    nonce = os.urandom(12)
    ciphertext = encrypt_ChaCha20(msg,secretKey,nonce)
    AlicePubKey = AlicePrivKey * curve.g
    return (ciphertext, AlicePubKey,nonce)


def decrypt_ECC(encryptedMsg, privKey):
    (ciphertext,AlicePubKey,nonce) = encryptedMsg
    sharedECCKey = privKey * AlicePubKey
    secretKey = ecc_point_to_256_bit_key(sharedECCKey)
    decrypted = decrypt_ChaCha20(secretKey,ciphertext,nonce)
    return decrypted


msg = b'I really want a border collie'
print("original msg:", msg)
print("\n")
BobPrivKey = secrets.randbelow(curve.field.n)
BobPubKey = BobPrivKey * curve.g

encryptedMsg = encrypt_ECC(msg, BobPubKey)
encryptedMsgObj = {
    'ciphertext': binascii.hexlify(encryptedMsg[0]),
    'AlicePubKey': hex(encryptedMsg[1].x) + hex(encryptedMsg[1].y % 2)[2:],
    'nonce': encryptedMsg[2]
}

print("encrypted msg:", encryptedMsgObj)

print("\n")
decryptedMsg = decrypt_ECC(encryptedMsg, BobPrivKey)
print("decrypted msg:", decryptedMsg)  



original msg: b'I really want a border collie'


secret key: b'f0b2c363c7ac4eede8d76df9d61a02385730dc43d4ae741b9e55613354580b11'
encrypted msg: {'ciphertext': b'f33a382bb15b6b4d04eebbdb1f42009f2143a2b2cb38408fc3298458d2b5889a7e4a2159376b3f72a73802138d', 'AlicePubKey': '0x24e7d268db6b4bd691db7c267386c5e6b04a7050d6e61a546e27dcd71bc1d1be0', 'nonce': b"\xcb >o\x91'P\x94\xe4Jdj"}


decrypted msg: bytearray(b'I really want a border collie')
